In [ ]:

%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

%reload_ext autoreload
%autoreload 2

from glob import glob
from pathlib import Path
import pandas as pd
from tqdm.auto import tqdm
from IPython.display import HTML, clear_output


def display_video(p, width='width=640'):
    html = f"""<video src = "{p}" {width} controls autoplay/>"""
    display(HTML(html))
    
mp4s = sorted(glob('khd/clip/*.mp4'))
mp4s = {Path(e).stem:e for e in mp4s}

anchors = sorted(glob('./khd/df_info/df_anchor_i/*.pickle'))
anchors = {Path(e).stem: e for e in anchors}

ppys = sorted(glob('./khd/preprocess/*/df_fan.pickle'))
ppys = {Path(e).parent.name:Path(e).parent for e in ppys}

print(f'anchors:{len(anchors)}, mp4s:{len(mp4s)}, ppys:{len(ppys)}')


In [ ]:
# 프리프로세싱되지 않은 pickle list 
errors = set(anchors.keys()) - set(ppys.keys()) 
errors = sorted(list(errors))
print(len(errors))

#list(ppys.keys())[0], ppys[list(ppys.keys())[0]]

In [ ]:
from pathlib import Path
import pdb

# 프리프로세싱되지 않은 pickle list 하나하나 살펴보기 
#for name in errors:
#    print(name)
#    df = pd.read_pickle(anchors[name])
#    display(df)
#    
#    print(mp4s[name[:-4]])
#    display_video(mp4s[name[:-4]])
#    
#    print(name)
#    assert(Path(mp4s[name[:-4]]).exists())
#    if input() == 'q':
#        break
#    clear_output(wait=True)

In [ ]:
from utils import crop_with_fan as cwf
from utils import face_finder as ff

l = list(ppys.items())
name, clip = l[0]
print(name, clip)

meta = ff.video_meta(mp4s[name[:-4]])
clip, meta['fps']

In [ ]:
from pathlib import Path
import pdb
import imageio
import cv2
import numpy as np
import imageio_ffmpeg
import os



In [ ]:
# 문제 없는지 일일이 보면서 검사하기
import os
from pathlib import Path
#if Path('checked.csv').exists():  
#    os.remove('checked.csv')

for i, (name, clip) in enumerate(ppys.items()):
    print('i:', i)
    try:
        df = pd.read_csv('checked_template_fhd.csv')
    except:
        df = pd.DataFrame([], columns=['clip', 'status'])
    
    if name in df['clip'].values:
        continue
    
    display(df.tail(3))
    
    cs = pd.read_pickle(clip/'df_fan.pickle')['cropped_size'].values[0]
    
    display_video(clip/'debug.mp4', width='width=640')
    print(f'{(len(df), i)}/{len(ppys)}: name({name}), cropped_size({cs}px)')
    c = input()
    if c == 'q':
        break
        
    checks = {'y':'yes', 'n':'no'}
    if c in checks:
        status = checks[c]
    else:
        status = None 
        
    df = df.append({'clip':name, 'status': status}, ignore_index=True)
    
    df.to_csv('checked_220208.csv', index=False)
    clear_output(wait=True)

In [ ]:
df['status'].value_counts(dropna=False)

In [ ]:
""" 0으로 시작하지 않는 영상이 있는지 검사"""
clips = glob('preprocessed_ytn/*')

len_zeros = []
for clip in clips:
    name = Path(clip).name
    pngs = sorted(glob(clip + '/*.png'))
    if len(pngs) == 0:
        len_zeros.append(name)
        continue
    min_idx = int(pngs[0].split('/')[-1].split('_')[0])
    if min_idx == 0:
        continue
    print(name, min_idx)
    
print('\nlen_zeros:')
for e in len_zeros:
    print(e)